### Libraries and Imports

In [1]:
import pandas as pd
from google_play_scraper import reviews_all, reviews, Sort
from pathlib import Path
import time
from datetime import timezone

### 1.1 - Initial Data Scraping → Creation of Initial DataFrame

Data was collected from the Google Play Store using the `google_play_scraper` library. The analysis focused on six major UK retail banks: Santander UK, HSBC, Lloyds, Barclays, Revolut, and Monzo. Although this selection does not encompass the entire UK retail banking landscape, it provides a representative overview of both traditional banks and digital-native banks, such as Revolut and Monzo.

The objective was to obtain individual reviews for each app, including attributes such as score, review text, date, replies, and other relevant metadata. These reviews serve as the input for subsequent sentiment analysis and categorization of user satisfaction and dissatisfaction drivers. 

A bootstrap function, `scrape_reviews_bootstrap()`, was employed to fetch all reviews and generate the initial DataFrame efficiently. For subsequent updates, the `scrape_reviews()` function is utilized to retrieve only new reviews that were not present in the previously created DataFrame.

In [3]:
def scrape_reviews_bootstrap():
    apps = {
        "Santander": "uk.co.santander.santanderUK",
        "HSBC": "uk.co.hsbc.hsbcukmobilebanking",
        "Lloyds": "com.grppl.android.shell.CMBlloydsTSB73",
        "Barclays": "com.barclays.android.barclaysmobilebanking",
        "Revolut": "com.revolut.revolut",
        "Monzo": "co.uk.getmondo",
    }

    rows = []
    for app_name, app_id in apps.items():
        start = time.time()
        try:
            batch = reviews_all(
                app_id,
                sleep_milliseconds=100,
                lang="en",
                country="gb"
            )
            for r in batch:
                dt = r.get("at")
                # ensure tz-aware UTC for compatibility with the incremental function
                if dt is not None:
                    if getattr(dt, "tzinfo", None) is None:
                        dt = dt.replace(tzinfo=timezone.utc)
                    else:
                        dt = dt.astimezone(timezone.utc)

                rows.append({
                    "app_name": app_name,
                    "app_id": app_id,                
                    "reviewId": r.get("reviewId"),    
                    "user_name": r.get("userName"),
                    "score": r.get("score"),
                    "text": r.get("content"),
                    "date": pd.Timestamp(dt) if dt is not None else pd.NaT,
                    "thumbs_up": r.get("thumbsUpCount"),
                    "Reply": r.get("replyContent"),
                    "Reply_Date": r.get("repliedAt"),
                    "App_Version": r.get("appVersion"),
                })
            print(f"✅ Fetched {len(batch)} reviews for {app_name} in {time.time()-start:.2f}s")
        except Exception as e:
            print(f"Error fetching reviews for {app_name}: {e}")
        time.sleep(1.0)

    df = pd.DataFrame(rows)

    # Normalize schema + de-dupe so future runs are smooth
    EXPECTED_COLS = [
        "app_name","app_id","reviewId","user_name","score","text","date",
        "thumbs_up","Reply","Reply_Date","App_Version"
    ]
    for c in EXPECTED_COLS:
        if c not in df.columns:
            df[c] = pd.NA
    df = df[EXPECTED_COLS]
    if "reviewId" in df.columns:
        df = df.drop_duplicates(subset=["reviewId"]).reset_index(drop=True)
    else:
        df = df.drop_duplicates().reset_index(drop=True)

    save_path = Path("D:/Digital-Banking-Dashboard/assets/intermediate_dfs/df_raw.parquet")
    save_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(save_path, index=False)
    print(f"✅ Bootstrap dataset saved → {save_path} (rows: {len(df)})")
    return df


In [4]:
df_raw = scrape_reviews_bootstrap()

✅ Fetched 80171 reviews for Santander in 38.17s
✅ Fetched 39524 reviews for HSBC in 20.05s
✅ Fetched 150762 reviews for Lloyds in 79.47s
✅ Fetched 268074 reviews for Barclays in 152.89s
✅ Fetched 265788 reviews for Revolut in 149.60s
✅ Fetched 22586 reviews for Monzo in 10.28s
✅ Bootstrap dataset saved → D:\Digital-Banking-Dashboard\assets\intermediate_dfs\df_raw.parquet (rows: 826905)


Display the raw DataFrame shape, summary information, and count of reviews per app.

In [5]:
print(f"df shape: {df_raw.shape}")
print('----------------------------')
print(df_raw.info())
print('----------------------------')
display(df_raw['app_name'].value_counts())

df shape: (826905, 11)
----------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826905 entries, 0 to 826904
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   app_name     826905 non-null  object             
 1   app_id       826905 non-null  object             
 2   reviewId     826905 non-null  object             
 3   user_name    826905 non-null  object             
 4   score        826905 non-null  int64              
 5   text         826780 non-null  object             
 6   date         826905 non-null  datetime64[ns, UTC]
 7   thumbs_up    826905 non-null  int64              
 8   Reply        114788 non-null  object             
 9   Reply_Date   114788 non-null  datetime64[ns]     
 10  App_Version  759394 non-null  object             
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), int64(2), object(7)
memory usage: 69.4+ MB
None
-----------------------

app_name
Barclays     268074
Revolut      265788
Lloyds       150762
Santander     80171
HSBC          39524
Monzo         22586
Name: count, dtype: int64

### 1.2 - Retrieval of New Reviews → DataFrame Update

`scrape_reviews()` function to update dataframe with new reviews.

In [6]:
def scrape_reviews():

    apps = {
        "Santander": "uk.co.santander.santanderUK",
        "HSBC": "uk.co.hsbc.hsbcukmobilebanking",
        "Lloyds": "com.grppl.android.shell.CMBlloydsTSB73",
        "Barclays": "com.barclays.android.barclaysmobilebanking",
        "Revolut": "com.revolut.revolut",
        "Monzo": "co.uk.getmondo"
    }

    save_path = Path("D:/Digital-Banking-Dashboard/assets/intermediate_dfs/df_raw.parquet")
  
    # Load existing data
    df_existing = pd.read_parquet(save_path)
    
    # Track seen reviews to avoid duplicates
    seen_ids = set(df_existing.get("reviewId", pd.Series(dtype=str)).dropna().unique())
    
    # Get latest date per app for incremental scraping
    latest_by_app = df_existing.groupby("app_name")["date"].max().to_dict()

    new_rows = []
    
    # Fetch new reviews for each app
    for app_name, app_id in apps.items():
        start = time.time()

        # Get cutoff date for this app
        latest_date = latest_by_app.get(app_name).tz_convert("UTC")

        token, keep_fetching, pages = None, True, 0

        # Paginate through reviews
        while keep_fetching:
            batch, token = reviews(
                app_id,
                lang="en",
                country="gb",
                sort=Sort.NEWEST,
                count=200,
                continuation_token=token,
            )
            pages += 1

            for r in batch:
                rid, rdate = r["reviewId"], r["at"].astimezone(timezone.utc)
                
                # Stop when reaching old reviews
                if rdate <= latest_date:
                    keep_fetching = False
                    break
                    
                # Skip duplicates
                if rid in seen_ids:
                    continue

                # Store new review
                new_rows.append({
                    "app_name": app_name,
                    "app_id": app_id,
                    "reviewId": rid,
                    "user_name": r.get("userName"),
                    "score": r.get("score"),
                    "text": r.get("content"),
                    "date": pd.Timestamp(rdate),
                    "thumbs_up": r.get("thumbsUpCount"),
                    "Reply": r.get("replyContent"),
                    "Reply_Date": r.get("repliedAt"),
                    "App_Version": r.get("appVersion"),
                })
                seen_ids.add(rid)

            if token is None:
                break

        # Log results
        app_new_count = sum(1 for x in new_rows if x['app_name'] == app_name)
        print(f"✅ {app_name}: +{app_new_count} new | {pages} page(s) in {time.time()-start:.2f}s")
        
        time.sleep(1.0)

    # Update dataset with new reviews
    df_out = df_existing

    if new_rows:
        df_new = pd.DataFrame(new_rows)
        
        # Combine and deduplicate
        df_out = (pd.concat([df_existing, df_new], ignore_index=True)
                    .drop_duplicates(subset=["reviewId"])
                    .reset_index(drop=True))
        
        df_out.to_parquet(save_path, index=False)
        print(f"✅ Saved {len(df_new)} new reviews (total {len(df_out)}) → {save_path}")
    else:
        print("ℹ️ No new reviews found.")

    return df_out

### 1.3 - Data Understanding

In [7]:
df_raw = scrape_reviews()

✅ Santander: +0 new | 1 page(s) in 0.38s
✅ HSBC: +0 new | 1 page(s) in 0.23s
✅ Lloyds: +0 new | 1 page(s) in 0.23s
✅ Barclays: +0 new | 1 page(s) in 0.27s
✅ Revolut: +0 new | 1 page(s) in 0.27s
✅ Monzo: +0 new | 1 page(s) in 0.19s
ℹ️ No new reviews found.


In [8]:
print(f"df shape: {df_raw.shape}")
print('----------------------------')
print(df_raw.info())
print('----------------------------')
display(df_raw['app_name'].value_counts())

df shape: (826905, 11)
----------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826905 entries, 0 to 826904
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   app_name     826905 non-null  object             
 1   app_id       826905 non-null  object             
 2   reviewId     826905 non-null  object             
 3   user_name    826905 non-null  object             
 4   score        826905 non-null  int64              
 5   text         826780 non-null  object             
 6   date         826905 non-null  datetime64[ns, UTC]
 7   thumbs_up    826905 non-null  int64              
 8   Reply        114788 non-null  object             
 9   Reply_Date   114788 non-null  datetime64[ns]     
 10  App_Version  759394 non-null  object             
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), int64(2), object(7)
memory usage: 69.4+ MB
None
-----------------------

app_name
Barclays     268074
Revolut      265788
Lloyds       150762
Santander     80171
HSBC          39524
Monzo         22586
Name: count, dtype: int64

Data was successfully collected and stored in the `df_raw.parquet` file, comprising **826,905 reviews**.

The distribution of reviews per app is as follows:

- **Barclays:** 268,074  
- **Revolut:** 265,788  
- **Lloyds:** 150,762  
- **Santander:** 80,171  
- **HSBC:** 39,524  
- **Monzo:** 22,586  

It should be noted that, despite the use of the `reviews_all` method, the **number of retrieved reviews is lower than the total number reported by the Google Play Store** for each app. This discrepancy can be attributed to several factors, including the following:

1. **Publicly accessible reviews only** – The `reviews_all` method retrieves only reviews that are publicly accessible. Some reviews may be hidden, restricted by location or language, or removed over time.

2. **Country and language filters** – Data collection was limited to English reviews from UK-based users. Reviews in other languages or from users in other countries, particularly for global apps such as Revolut, were not included.

3. **Scraping limitations** – Google Play’s paginated interface does not provide all reviews in a single request. Although `reviews_all` loops through pages to retrieve reviews, a limit is typically reached. Additionally, Google may intentionally restrict the amount of content accessible via unofficial scraping tools.

During the most recent update, **no new reviews were found** for any of the six apps, indicating that the dataset is current.

Despite these constraints, **the collected dataset is sufficient to extract meaningful insights and derive conclusions**, as the primary objective of this project is to analyze sentiment and identify key topics for each app, rather than to account for the absolute total number of reviews.